# Optimizing the parameters of the Lorenz system
This notebook explores PyTorch's autograd to optimize Lorenz's nonlinear dynamical problem. A three-dimensional state vector evolves nonlinearly in this coupled system and the solution depends on three parameters. The notebook covers and compares several different approaches to tune the parameters: optimizing the parameters using gradient-based optimization based on a single scenario, and "online" optimization of the parameters for that same scenario.

## Lorenz equations and numerical discretization


In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

def lorenz_torch(state: torch.Tensor, sigma: torch.Tensor, rho: torch.Tensor, beta: torch.Tensor) -> torch.Tensor:
    """
    Compute the Lorenz system derivatives.
    
    Args:
        state: tensor of shape (3,) or (..., 3), representing (x, y, z)
        sigma: torch tensor scalar (can require gradients)
        rho: torch tensor scalar (can require gradients)
        beta: torch tensor scalar (can require gradients)
    
    Returns:
        torch.Tensor: derivatives of same shape as state
    """
    x, y, z = state[..., 0], state[..., 1], state[..., 2]

    dx = sigma * (y - x)
    dy = x * (rho - z) - y
    dz = x * y - beta * z

    return torch.stack([dx, dy, dz], dim=-1)

def euler_step_torch(state: torch.Tensor, dt: float, sigma: torch.Tensor, rho: torch.Tensor, beta: torch.Tensor) -> torch.Tensor:
    """
    Perform one Euler integration step.
    
    Args:
        state: torch tensor of shape (3,) or (..., 3)
        dt: float, time step size
        sigma: torch tensor scalar
        rho: torch tensor scalar
        beta: torch tensor scalar
    
    Returns:
        torch.Tensor: updated state
    """
    return state + dt * lorenz_torch(state, sigma, rho, beta)

Let's write a function that can advance an initial state for a certain number of time steps.

In [ ]:
def run_lorenz(state0, t0, steps, dt, sigma, rho, beta):
    state = state0
    trajectory = []
    time = []

    for i in range(steps):
        state = euler_step_torch(state, dt, sigma, rho, beta)
        trajectory.append(state)
        time.append(torch.tensor(t0 + (i + 1) * dt))

    return torch.stack(time), torch.stack(trajectory)

## Gradient-based optimization of a past scenario
Let's now run a gradient-based optimization for one scenario. That is, let's define the "true" trajectory for one initial state and let's iterate over many parameter configurations (i.e., trials) to minimize the distance to the true trajectory (i.e., loss).

The approach is to define the parameters first with initial guesses and to set `requires_grad=True` such that the computational graph will be computed to enable subsequent gradient-based incremental changes to the parameter set. 

The function below takes in the number of total time steps to use to solve the optimization. Each trajectory is currently covered by 100 time steps, such that a total of 10000 time steps would equate to 100 trials.

In [ ]:
def scenario_optimization(state0, num_tot_steps=10000):
    # Set number of trials 
    num_trials = int(num_tot_steps / 100)


    # Initial parameter choices
    sigma = torch.tensor(8.0, requires_grad=True)
    rho   = torch.tensor(20.0, requires_grad=True)
    beta  = torch.tensor(3.0, requires_grad=True)

    true_time, true_traj = get_observations(state0)

    optimizer = torch.optim.Adam([sigma, rho, beta], lr=1e-2)

    # Store trajectories and losses for visualization
    stored_trajectories = []
    stored_losses = []

    for trial in range(num_trials):
        optimizer.zero_grad()

        pred_time, pred_traj = run_lorenz(state0, t0=0, steps=100, dt=0.01, sigma=sigma, rho=rho, beta=beta)
        loss = torch.mean((pred_traj - true_traj) ** 2)

        loss.backward()   # <-- AUTOGRAD HERE
        optimizer.step()

        if trial % 20 == 0 or trial == num_trials - 1:
            print(f"Trial {trial:4d} | Loss={loss.item():.6f} "
                f"sigma={sigma.item():.3f}, rho={rho.item():.3f}, beta={beta.item():.3f}")
            # Store trajectory and loss for plotting
            stored_trajectories.append((trial, pred_traj.detach().clone()))
            stored_losses.append((trial, loss.item()))
    return [traj for _, traj in stored_trajectories], [loss for _, loss in stored_losses]



Now let's run this from an initial state.

In [ ]:
state0 = torch.tensor([1.0, 1.0, 1.0])
trajectories, losses = scenario_optimization(state0, num_tot_steps=200000)

In [ ]:
# Visualize the trajectories
fig = plt.figure(figsize=(14, 6))

# 3D plot of all trajectories
ax1 = fig.add_subplot(121, projection='3d')

# Plot the true (observed) trajectory
true_traj_np = true_traj.detach().numpy()
ax1.plot(true_traj_np[:, 0], true_traj_np[:, 1], true_traj_np[:, 2], 
         'o-', linewidth=2, markersize=4, label='Observed', color='black')

# Plot stored trajectories with color gradient representing iterations
colors = plt.cm.viridis(np.linspace(0, 1, len(stored_trajectories)))
for i, trial_traj in enumerate(stored_trajectories):
    trial, traj = trial_traj
    traj_np = traj.numpy()
    ax1.plot(traj_np[:, 0], traj_np[:, 1], traj_np[:, 2], 
             alpha=0.3, linewidth=0.5, color=colors[i])

ax1.set_xlabel('X')
ax1.set_ylabel('Y')
ax1.set_zlabel('Z')
ax1.set_title('Lorenz Trajectories Over Optimization')
ax1.legend()

# Loss curve
ax2 = fig.add_subplot(122)
trials, losses = zip(*stored_losses)
ax2.plot(trials, losses, 'b-', marker='o', markersize=4)
ax2.set_xlabel('Trial')
ax2.set_ylabel('Loss')
ax2.set_title('Loss During Optimization')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nTotal trajectories plotted: {len(stored_trajectories)} (plus the observed one)")
print(f"Final loss: {losses[-1]:.6f}")

## Online parameter estimation in real time
This works fine. But one problem is that we can carry out this type of optimization only if we have observations for the full trajectory, which for some systems might only be available for past dates. That is, the optimization can only occur for past scenarios. An alternative tested next is to assume that observations trickle in in real-time. In that case, we can adjust the parameters in real-time as we move forward to arrive at an optimal parameter configuration over time. In this sense, we use the parameters to nudge the system toward the observations over time. 

In [ ]:
# Reset parameters for online estimation
sigma_online = torch.tensor(8.0, requires_grad=True)
rho_online   = torch.tensor(20.0, requires_grad=True)
beta_online  = torch.tensor(3.0, requires_grad=True)

# Starting state
current_state = state0.clone()
current_time = 0.0

# Extract times and states from true trajectory
true_times = true_time
true_states = true_traj

# Store results for visualization
online_trajectories = []
online_losses = []
online_params = []

# Number of optimization trials per observation
# In the previous optimization we ran 2000 trials, each 100 time steps
# Let's keep the total number of simulated steps the same such that per 
# time step we optimize with n_trials_online trials
n_trials_online = int(25000 / 100)

# Process each observation
for obs_idx in range(len(true_states)):
    target_time = true_times[obs_idx].item()
    dt_to_target = target_time - current_time
    num_steps_to_target = 1 #int(dt_to_target / 0.01)  # Currently one step from previous observation to next
    observed_state = true_states[obs_idx]
    target_time = num_steps_to_target * 0.01 + current_time
    # Optimizer for this online step
    optimizer_online = torch.optim.Adam([sigma_online, rho_online, beta_online], lr=1e-2)
    
    best_loss = float('inf')
    best_pred_state = None
    
    # Run N optimization trials for this observation
    for trial in range(n_trials_online):
        optimizer_online.zero_grad()
        
        # Advance from current state to next observation point
        next_state = run_lorenz(current_state, current_time, num_steps_to_target, 0.01, sigma_online, rho_online, beta_online)[1][-1]
        
        # Compute loss against observed state
        loss = torch.mean((next_state - observed_state) ** 2)
        
        loss.backward()
        optimizer_online.step()
        
        # Track best solution
        if loss.item() < best_loss:
            best_loss = loss.item()
            best_pred_state = next_state.detach().clone()
    
    # Update current state and time with optimal prediction
    current_state = best_pred_state.detach().requires_grad_(True)
    current_time = target_time
    
    # Store results
    online_trajectories.append((obs_idx, current_state.detach().clone()))
    online_losses.append((obs_idx, best_loss))
    online_params.append({
        'obs_idx': obs_idx,
        'sigma': sigma_online.item(),
        'rho': rho_online.item(),
        'beta': beta_online.item(),
        'loss': best_loss
    })
    
    if obs_idx % 1 == 0 or obs_idx == len(true_traj) - 1:
        print(f"Obs {obs_idx:3d} | Loss={best_loss:.6f} | "
              f"sigma={sigma_online.item():.3f}, rho={rho_online.item():.3f}, beta={beta_online.item():.3f}")

# Compute final loss from all concatenated online trajectories
final_online_traj = torch.stack([traj for _, traj in online_trajectories])
final_loss = torch.mean((final_online_traj - true_states) ** 2).item()
print(f"Final online estimation loss: {final_loss:.6f}")

print(f"\nOnline estimation complete. Processed {len(true_traj)-1} observations.")

In [ ]:
# Visualize the trajectories
fig = plt.figure(figsize=(14, 6))

# 3D plot of all trajectories
ax1 = fig.add_subplot(121, projection='3d')

# Plot the true (observed) trajectory
true_traj_np = true_traj.detach().numpy()
ax1.plot(true_traj_np[:, 0], true_traj_np[:, 1], true_traj_np[:, 2], 
         'o-', linewidth=2, markersize=4, label='Observed', color='black')
final_online_traj_np = final_online_traj.detach().numpy()
ax1.plot(final_online_traj_np[:, 0], final_online_traj_np[:, 1], final_online_traj_np[:, 2], 
         'o-', linewidth=2, markersize=4, label='Estimated', color='red')


ax1.set_xlabel('X')
ax1.set_ylabel('Y')
ax1.set_zlabel('Z')
ax1.set_title('Lorenz Trajectories Over Optimization')
ax1.legend()


In [ ]:
def get_observations(state0):
    # Fake "observed" data (normally you'd load real data)
    with torch.no_grad():
        true_time, true_traj = run_lorenz(
            state0,
            t0=0,
            steps=100,
            dt=0.01,
            sigma=torch.tensor(10.0),
            rho=torch.tensor(28.0),
            beta=torch.tensor(8/3),
        )
        return true_time, true_traj

In [ ]:
def online_optimization(state0, num_tot_steps=200000):
    # Reset parameters for online estimation
    sigma_online = torch.tensor(8.0, requires_grad=True)
    rho_online   = torch.tensor(20.0, requires_grad=True)
    beta_online  = torch.tensor(3.0, requires_grad=True)

    true_time, true_traj = get_observations(state0)

    # Starting state
    current_state = state0.clone()
    current_time = 0.0

    # Extract times and states from true trajectory
    true_times = true_time
    true_states = true_traj

    # Store results for visualization
    online_trajectories = []
    online_losses = []
    online_params = []

    # Number of optimization trials per observation
    # In the previous optimization we ran 2000 trials, each 100 time steps
    # Let's keep the total number of simulated steps the same such that per 
    # time step we optimize with n_trials_online trials
    n_trials_online = int(num_tot_steps / 100)

    # Process each observation
    for obs_idx in range(len(true_states)):
        target_time = true_times[obs_idx].item()
        dt_to_target = target_time - current_time
        num_steps_to_target = 1 #int(dt_to_target / 0.01)  # Currently one step from previous observation to next
        observed_state = true_states[obs_idx]
        target_time = num_steps_to_target * 0.01 + current_time
        # Optimizer for this online step
        optimizer_online = torch.optim.Adam([sigma_online, rho_online, beta_online], lr=1e-2)
        
        best_loss = float('inf')
        best_pred_state = None
        
        # Run N optimization trials for this observation
        for trial in range(n_trials_online):
            optimizer_online.zero_grad()
            
            # Advance from current state to next observation point
            next_state = run_lorenz(current_state, current_time, num_steps_to_target, 0.01, sigma_online, rho_online, beta_online)[1][-1]
            
            # Compute loss against observed state
            loss = torch.mean((next_state - observed_state) ** 2)
            
            loss.backward()
            optimizer_online.step()
            
            # Track best solution
            if loss.item() < best_loss:
                best_loss = loss.item()
                best_pred_state = next_state.detach().clone()
        
        # Update current state and time with optimal prediction
        current_state = best_pred_state.detach().requires_grad_(True)
        current_time = target_time
        
        # Store results
        online_trajectories.append((obs_idx, current_state.detach().clone()))
        online_losses.append((obs_idx, best_loss))
        online_params.append({
            'obs_idx': obs_idx,
            'sigma': sigma_online.item(),
            'rho': rho_online.item(),
            'beta': beta_online.item(),
            'loss': best_loss
        })
        
        if obs_idx % 1 == 0 or obs_idx == len(true_traj) - 1:
            print(f"Obs {obs_idx:3d} | Loss={best_loss:.6f} | "
                f"sigma={sigma_online.item():.3f}, rho={rho_online.item():.3f}, beta={beta_online.item():.3f}")

    # Compute final loss from all concatenated online trajectories
    final_online_traj = torch.stack([traj for _, traj in online_trajectories])
    final_loss = torch.mean((final_online_traj - true_states) ** 2).item()
    print(f"Final online estimation loss: {final_loss:.6f}")

    print(f"\nOnline estimation complete. Processed {len(true_traj)-1} observations.")
    return final_online_traj, final_loss

def scenario_optimization(state0, num_tot_steps=200000):
    # Set number of trials 
    num_trials = int(num_tot_steps / 100)


    # Initial parameter choices
    sigma = torch.tensor(8.0, requires_grad=True)
    rho   = torch.tensor(20.0, requires_grad=True)
    beta  = torch.tensor(3.0, requires_grad=True)

    true_time, true_traj = get_observations(state0)

    optimizer = torch.optim.Adam([sigma, rho, beta], lr=1e-2)

    # Store trajectories and losses for visualization
    stored_trajectories = []
    stored_losses = []

    for trial in range(num_trials):
        optimizer.zero_grad()

        pred_time, pred_traj = run_lorenz(state0, t0=0, steps=100, dt=0.01, sigma=sigma, rho=rho, beta=beta)
        loss = torch.mean((pred_traj - true_traj) ** 2)

        loss.backward()   # <-- AUTOGRAD HERE
        optimizer.step()

        if trial % 20 == 0 or trial == num_trials - 1:
            print(f"Trial {trial:4d} | Loss={loss.item():.6f} "
                f"sigma={sigma.item():.3f}, rho={rho.item():.3f}, beta={beta.item():.3f}")
            # Store trajectory and loss for plotting
            stored_trajectories.append((trial, pred_traj.detach().clone()))
            stored_losses.append((trial, loss.item()))
    return [traj for _, traj in stored_trajectories], [loss for _, loss in stored_losses]



Now let's compare the two methods, the scenario optimization and the online-optimization. We can use the same number of total time steps used during the optimization and compute the resulting loss from both methods. Let's run this and then plot the loss as a function of the total number of time steps.

In [ ]:
# Initial condition
state0 = torch.tensor([1.0, 1.0, 1.0])
output = []
for num_tot_steps in [int(x) for x in np.logspace(np.log10(2000), np.log10(200000), 10)]:
    output.append((num_tot_steps, scenario_optimization(state0, num_tot_steps=num_tot_steps), online_optimization(state0, num_tot_steps=num_tot_steps)))
# Extract data for plotting
steps = [item[0] for item in output]
scenario_losses = [item[1] for item in output]
online_losses = [item[2] for item in output]

# Create the plot
plt.figure(figsize=(10, 6))
plt.plot(steps, scenario_losses, 'b-o', label='Scenario Optimization', linewidth=2, markersize=6)
plt.plot(steps, online_losses, 'r-s', label='Online Optimization', linewidth=2, markersize=6)
plt.xlabel('Number of Total Steps')
plt.ylabel('Loss')
plt.title('Comparison of Optimization Methods vs Total Steps')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()


We can see that the online optimization, which optimizes each timestep, performs better as soon as the number of steps is reasonably large. For very large numbers the two approaches become identical. The way I interpret this is that the online optimization per timestep optimizes smaller problems with less complicated loss functions. Therefore, the overall number of steps needed to optimize the full trajectory ends up being smaller than for the scenario optimization approach that optimizes the full trajectory.

The real-time online optimization of course only makes sense if the optimization between observations can get carried out faster than real-time. That is, as a new observation comes in, we need to advance the state from the time the last observation was obtained, but not once, but N-trial times.